##What is a relational Database?

"A database is a means of storing information in such a way that information can be retrieved from it. In simplest terms, a relational database is one that presents information in tables with rows and columns. A table is referred to as a relation in the sense that it is a collection of objects of the same type (rows). Data in a table can be related according to common keys or concepts, and the ability to retrieve related data from a table is the basis for the term relational database."

https://docs.oracle.com/javase/tutorial/jdbc/overview/database.html

##How does a database compare to a spreadsheet?

###Table

Each table is like a separate spreadsheet (or spreadsheet tab).


In [26]:
print ("The fruit table")
import sqlite3 #sqlite3 is a database library
conn = sqlite3.connect(':memory:') #create and get connection to in-memory temporary database
c = conn.cursor() #cursors are used for executing commands and selecting data from database

# Create table
c.execute('''CREATE TABLE fruit
             ( fruit_id   INTEGER PRIMARY KEY
             , fruit_name TEXT
             , price      NUMERIC)''')

# Insert data

c.execute("INSERT INTO fruit VALUES (1, 'apple', 0.88)")
c.execute("INSERT INTO fruit VALUES (2, 'banana', 0.34)")
c.execute("INSERT INTO fruit VALUES (3, 'orange', 1.07)")
c.execute("INSERT INTO fruit VALUES (4, 'forbidden', NULL)")
c.execute("INSERT INTO fruit VALUES (0, '', 0)")
c.execute("INSERT INTO fruit VALUES (NULL, NULL, NULL)")

# Save (commit) the changes
conn.commit()

#print table
from prettytable import from_db_cursor 
c.execute("SELECT * FROM fruit ORDER BY fruit_id") 
pt = from_db_cursor(c)
print (pt)

The fruit table
+----------+------------+-------+
| fruit_id | fruit_name | price |
+----------+------------+-------+
|    0     |            |   0   |
|    1     |   apple    |  0.88 |
|    2     |   banana   |  0.34 |
|    3     |   orange   |  1.07 |
|    4     | forbidden  |  None |
|    5     |    None    |  None |
+----------+------------+-------+


###Column

Unlike spreadsheet columns, database columns each have a specific type and can only store certain types of data. The most common types include: TEXT, NUMERIC, INTEGER and REAL. Unlike spreadsheets, columns cannot be merged. In the above example, the columns are: `fruit_id`, `fruit_name` and `price`. The `fruit_id` column is a primary key which means values for it cannot be repeated so that a value uniquely identifies its row. 

see https://www.sqlite.org/datatype3.html

###Row

Like spreadsheets, tables are organized into rows of data. The example above has 6 rows of data.

###Field

A column for a particular row is called a field. If a field has no data, it contains the special value `NULL`. This is different than the empty string: "" and the number: 0. Primary keys usually cannot be NULL.

In [27]:
print ("Fruit with NULL price")
c.execute('''SELECT * FROM fruit 
             WHERE price is NULL 
             ORDER BY fruit_id''') 
pt = from_db_cursor(c)
print (pt)


Fruit with NULL price
+----------+------------+-------+
| fruit_id | fruit_name | price |
+----------+------------+-------+
|    4     | forbidden  |  None |
|    5     |    None    |  None |
+----------+------------+-------+


In [28]:
conn.close() #close connection when finished